In [187]:
import os
from io import StringIO
from textwrap import dedent
from typing import List

import pandas as pd
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
load_dotenv()

from langchain_core.pydantic_v1 import BaseModel, Field

os.environ["UNSTRUCTURED_API_KEY"]


'sPySfJ3bGk4Z9M9MC2iP9NGqmxuDMO'

In [195]:

html_text = """<table><thead><tr><th colspan="7">BCBST MEDICAL PLANS</th></tr><tr><th></th><th colspan="2">Blue Plan</th><th colspan="2">Orange Plan</th><th colspan="2">White Plan</th></tr></thead><tbody><tr><td></td><td>IN-NETWORK</td><td>OUT-OF- NETWORK</td><td>IN-NETWORK</td><td>OUT-OF- NETWORK</td><td colspan="2">IN-NETWORK OUT-OF- NETWORK</td></tr><tr><td>Tax Advantage Account Eligibility (See page 9 for details)</td><td>Health Savings</td><td>Account</td><td colspan="2">Health Savings Account</td><td colspan="2">Flexible Spending Account</td></tr><tr><td rowspan="2">Deductible Type</td><td colspan="2">Embedded Deductible</td><td colspan="2">Embedded Deductible</td><td colspan="2">Embedded Deductible</td></tr><tr><td colspan="2" rowspan="2">(See Glossary on page 26 for definitions) $4,000/ $4,000 $8,000 $8,000 $16,000</td><td colspan="2">(See Glossary on page 26 for definitions)</td><td colspan="2">(See Glossary on page 26 for definitions)</td></tr><tr><td>Annual Deductible Individual / Individual in a Family</td><td>$2,000 / $3,200 $4,000</td><td>$4,000 $8,000</td><td>$1,000/ $1,000 $2,000</td><td>$2,000 $4,000</td></tr><tr><td>Family Coinsurance (after deductible)</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td></tr><tr><td>Out-of-Pocket Maximum Type</td><td colspan="2">Embedded Out-of-Pocket Maximum</td><td colspan="2">Embedded Out-of-Pocket Maximum</td><td>Embedded Out-of-Pocket</td><td>Maximum</td></tr><tr><td rowspan="2">Out-of-Pocket Maximum (Includes copays, deductible, coinsurance)</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>$7,000 $14,000</td><td>$14,000 $28,000</td><td>$4,000 $8,000</td><td>$8,000 $16,000</td><td>$2,000 $4,000</td><td>$4,000 $8,000</td></tr><tr><td>Preventive Care</td><td>BCBST pays 100%</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 100%</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 100%</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Primary Care Office Visit</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>$40 Copay</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Premise Health Center</td><td colspan="2">$0 for a Preventive Visit $5 for a Sick Visit</td><td colspan="2">$0 for a Preventive Visit $5 for a Sick Visit</td><td colspan="2">$0 for a Preventive Visit $5 for a Sick Visit</td></tr><tr><td>Specialist Office Visit</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>$40 Copay</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Urgent Care Office Visit</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>$75 Copay</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Premise Virtual Care Visit</td><td colspan="2">$0/ Deductible Waived</td><td colspan="2">$0/ Deductible Waived</td><td colspan="2">$0/ Deductible Waived</td></tr><tr><td>Hospital - Inpatient</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Outpatient - Surgery</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td><td>BCBST pays 80% after deductible</td><td>BCBST pays 60% after deductible</td></tr><tr><td>Emergency Room</td><td colspan="2">BCBST pays 80% after deductible</td><td colspan="2">BCBST pays 80% after deductible</td><td colspan="2">BCBST pays 80% after deductible</td></tr></tbody></table>

"""
from IPython.core.display import HTML
HTML(html_text)
#table = pd.read_html(StringIO(html_text))
#table

In [183]:

def get_pinecone_retriever(
        index_name, 
        embedding,
        k
):
    vectorstore = PineconeVectorStore.from_existing_index(
        index_name=index_name, 
        embedding=embedding,
    )
    return vectorstore.as_retriever(search_kwargs={"k": k})

pinecone_index = "lp-hr-rag"
retriever = get_pinecone_retriever(
    index_name=pinecone_index, 
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    k=10
)

In [210]:
from enum import Enum
from textwrap import dedent
gpt_4o = init_chat_model(
    model="gpt-4o", 
    model_provider="openai", 
    temperature=0)

class Prompts(Enum):
    rag = dedent("""
    Answer the question based only on the following context. Cite the sources, page numbers, and urls of the information you used.
    
    Context: {context}
    
    Question: {question}
    """)


# Good one
prompt = ChatPromptTemplate.from_template(Prompts.rag.value)
def format_docs(docs: List[Document]) -> str:
    """Convert Documents to a single string.:""" 
    formatted = [
        (f"Source: {doc.metadata['file_name'].replace('-', ' ').replace('_', ' ')}\nURL: {doc.metadata['url']}"
         f"\nPage Number: {int(doc.metadata['page_number'])}\nContent: {doc.page_content if doc.metadata['category'] != 'Table' else doc.metadata['text_as_html']}")
        for doc in docs
    ]
    return "\n\n" + "\n\n".join(formatted)

rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()
     }
    | prompt
    | gpt_4o
    | StrOutputParser()
)

In [219]:
question = "Can I relocate while still at LP?" 
response = rag_chain.invoke(question)
print(response)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Yes, you can relocate while still at LP if you meet the eligibility criteria and receive the necessary approvals. According to the Relocation Policy, US or Canadian employees who transfer at LP’s request are eligible for relocation benefits based on the salary grade of the position they are transferring into (Source: Relocation Policy, https://lpcorp.com/, Page 1). For US employees, the relocation must involve a move where the distance between the employee’s former residence and new place of work is at least 50 miles greater than the distance between the employee’s former residence and former place of work (Source: Relocation Policy, https://lpcorp.com/, Page 1). For Canadian employees, the move must result in the employee living at least 40 km closer to the new work or business location (Source: Relocation Policy, https://lpcorp.com/, Page 1).

All signed offer letters containing relocation benefits must be sent to LP’s Relocation Manager, who will then authorize the move with the Com

In [102]:
[doc for doc in docs if doc.metadata['category'] == "Table"][0].metadata["text_as_html"]

'<table><tbody><tr><td>Qualifying Event</td><td>Coverage Change Allowed</td><td>Documentation Required</td></tr><tr><td>Marriage\n @</td><td>Add or remove coverage for employee and/ or dependents, change plans or tiers</td><td>Copy of marriage certificate</td></tr><tr><td>Birth or Adoption</td><td>Add coverage for employee and/or new child</td><td>Copy of birth certificate or adoption paperwork</td></tr><tr><td>Loss or Gain of Other + Coverage</td><td>Add or remove coverage for employee and/ or affected dependent</td><td>Documentation providing proof of loss or gain of coverage with effective date</td></tr></tbody></table>'

In [103]:
[doc for doc in docs if doc.metadata['page_number'] == 8]

[Document(page_content='Medical Overview', metadata={'filetype': 'application/pdf', 'page_number': 8, 'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'languages': 'eng', 'category': 'Title', 'url': 'https://lpcorp.com/', 'source': '2024-LP-Benefits-Guide---US-Salary-updated-V3'}),
 Document(page_content='Louisiana Pacific offers employees the choice of three medical plans through BCBST. See below for an overview of each plan and refer to the comparison chart on Medical Plan Comparison on page 7 to see how certain common services are covered.', metadata={'filetype': 'application/pdf', 'page_number': 8, 'parent_id': '5693ba5030365c1eac74440a5ed1742b', 'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'languages': 'eng', 'category': 'NarrativeText', 'url': 'https://lpcorp.com/', 'source': '2024-LP-Benefits-Guide---US-Salary-updated-V3'}),
 Document(page_content='Blue & Orange Plan', metadata={'filetype': 'application/pdf', 'page_number': 8, 'filename': '202

In [191]:
question = "Explain the different health plans available to salaried employees" 
response = rag_chain.stream(question)

print(response)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Based on the provided context, salaried employees at LP Corporation have access to three different health plans under the BCBST Medical Plans: the Blue Plan, the Orange Plan, and the White Plan. Here is a detailed explanation of each plan:

### Blue Plan
- **Tax Advantage Account Eligibility**: Health Savings Account (HSA)
- **Deductible Type**: Embedded Deductible
- **Annual Deductible**:
  - Individual: $4,000
  - Family: $8,000
- **Coinsurance (after deductible)**:
  - In-Network: You pay 20%, BCBST pays 80%
  - Out-of-Network: You pay 40%, BCBST pays 60%
- **Out-of-Pocket Maximum**:
  - Individual: $7,000
  - Family: $14,000
- **Preventive Care**: BCBST pays 100% in-network
- **Primary Care Office Visit**: BCBST pays 80% after deductible in-network
- **Specialist Office Visit**: BCBST pays 80% after deductible in-network
- **Urgent Care Office Visit**: BCBST pays 80% after deductible in-network
- **Premise Virtual Care Visit**: $0, deductible waived
- **Hospital - Inpatient**: BCBS

In [86]:
response

'The gift limit for LP employees and their immediate family members is $100. \n\nSource: LP Code of Conduct, Page Number: 48, URL: https://lpcorp.com/'

In [82]:
print(response['answer'])

TypeError: string indices must be integers, not 'str'

In [38]:
response['docs']

[Document(page_content="See LP's Anti-Corruption Policy", metadata={'category': 'NarrativeText', 'filename': 'LP-Code-of-Conduct.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 58.0, 'source': 'LP-Code-of-Conduct', 'url': 'https://lpcorp.com/'}),
 Document(page_content='Acting in LP’s Best Interests — Employees, Directors, and Officers of LP are expected to act in the best interests of the Company. This means that you should not use your position for private gain from customers, vendors, suppliers, or contractors.', metadata={'category': 'NarrativeText', 'filename': 'LP-Code-of-Conduct.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 44.0, 'parent_id': 'eb4791d05a3019a1330b37fb0f10e5bc', 'source': 'LP-Code-of-Conduct', 'url': 'https://lpcorp.com/'}),
 Document(page_content="We hope this Code has been of help to you in understanding your obligations and LP's expectations. Most of us want to do the right thing, and most of the time, it is cl

In [181]:



class cited_answer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: List[int] = Field(
        ...,
        description="The url of the SPECIFIC sources which justify the answer.",
    )

In [79]:
llm_with_tool = gpt_4o.bind_tools(
    [cited_answer],
    tool_choice="cited_answer",
)
question = "What is lp's mission?"
llm_with_tool.invoke(question)

NameError: name 'cited_answer' is not defined

In [176]:
response['answer']

'The mission of LP is to act with integrity and uphold the highest ethical standards.'

In [178]:
response

{'answer': 'The mission of LP is to act with integrity and uphold the highest ethical standards.',
 'docs': [Document(page_content='While LP’s Code cannot cover every situation you will encounter, its goal is to set high ethical standards of behavior, explain our responsibilities to the company and each other, direct you to more comprehensive policies on specific requirements, and provide a confidential avenue for reporting behavior that you believe contrary to our duties and culture.', metadata={'category': 'NarrativeText', 'filename': 'LP-Code-of-Conduct.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 5.0, 'parent_id': '4434b8c446883f4b1eba273406054a63', 'source': 'LP-Code-of-Conduct', 'url': 'some_url'}),
  Document(page_content='Tuition Assistance Program LP encourages and supports employee self- improvement through formal education so employees can enhance their job-related skills. LP will reimburse 80% of eligible expenses paid by the employee up to the IR

In [170]:
response

{'answer': 'The policy around document retention is not specified in the provided context.',
 'docs': [Document(page_content="See LP's Record Retention Policy", metadata={'category': 'NarrativeText', 'filename': 'LP-Code-of-Conduct.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 34.0, 'parent_id': 'bb03086915d56df2d45c56a10e3d8787', 'source': 'LP-Code-of-Conduct', 'url': 'some_url'}),
  Document(page_content='Documents and Records', metadata={'category': 'Title', 'filename': 'LP-Code-of-Conduct.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 34.0, 'parent_id': '896c0383080c5292cd2e1793b3194b0a', 'source': 'LP-Code-of-Conduct', 'url': 'some_url'}),
  Document(page_content='Retaining Required Records — LP is committed to effective document maintenance and records retention to protect its history, ensure that records are available to meet the Company’s business and legal needs, efficiently manage the use of space, minimize the cost of record

In [120]:
response['context'][0].metadata

{'category': 'NarrativeText',
 'filename': 'LP-Code-of-Conduct.pdf',
 'filetype': 'application/pdf',
 'languages': 'eng',
 'page_number': 5.0,
 'parent_id': '4434b8c446883f4b1eba273406054a63',
 'source': 'LP-Code-of-Conduct',
 'url': 'some_url'}

In [122]:
print(response['answer'].content)

LP's values include high ethical standards, mutual respect and trust, safety and health, responsible corporate citizenship, and environmental care.


'lp-hr'

In [354]:
[document.metadata['text_as_html'] if document.metadata['category'] == "Table" else document for document in documents]

KeyError: 'text_as_html'

In [347]:

[split.page_content for split in  all_splits[:10]][6]

'Individual in a Family</td><td colspan="2"></td><td>$2,000 / $3,200 $4,000</td><td>$4,000 $8,000</td><td>$1,000/ $1,000 $2,000</td><td>$2,000 $4,000</td></tr><tr><td>Family Coinsurance (after deductible)</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td><td>You Pay 20% BCBST Pays 80%</td><td>You Pay 40% BCBST Pays 60%</td></tr><tr><td>Out-of-Pocket Maximum Type</td><td colspan="2">Embedded Out-of-Pocket Maximum</td><td colspan="2">Embedded Out-of-Pocket Maximum</td><td>Embedded Out-of-Pocket</td><td>Maximum</td></tr><tr><td rowspan="2">Out-of-Pocket Maximum (Includes copays, deductible, coinsurance)</td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td></td><td>$7,000 $14,000</td><td>$14,000 $28,000</td><td>$4,000 $8,000</td><td>$8,000 $16,000</td><td>$2,000 $4,000</td><td>$4,000 $8,000</td></tr><tr><td>Preventive Care</td><td>BCBST pays 100%</td><td>BCBST pays 60% after'

In [349]:
[split.metadata['text_as_html'] for split in  all_splits[:10]][6]

'<table><thead><th colspan="7">BCBST MEDICAL PLANS</th></thead><thead><th></th><th colspan="2">Blue Plan</th><th colspan="2">Orange Plan</th><th colspan="2">White Plan</th></thead><tr><td></td><td>IN-NETWORK</td><td>OUT-OF- NETWORK</td><td>IN-NETWORK</td><td>OUT-OF- NETWORK</td><td colspan="2">IN-NETWORK OUT-OF- NETWORK</td></tr><tr><td>Tax Advantage Account Eligibility (See page 9 for details)</td><td>Health Savings</td><td>Account</td><td colspan="2">Health Savings Account</td><td colspan="2">Flexible Spending Account</td></tr><tr><td rowspan="2">Deductible Type</td><td colspan="2">Embedded Deductible</td><td colspan="2">Embedded Deductible</td><td colspan="2">Embedded Deductible</td></tr><tr><td></td><td colspan="2" rowspan="2">(See Glossary on page 26 for definitions) $4,000/ $4,000 $8,000 $8,000 $16,000</td><td colspan="2">(See Glossary on page 26 for definitions)</td><td colspan="2">(See Glossary on page 26 for definitions)</td></tr><tr><td>Annual Deductible Individual / Individu

In [328]:
[split.metadata['text_as_html'][:10] for split in  all_splits[:10]]

['<table><tr',
 '<table><tr',
 '<table><tr',
 '<table><th',
 '<table><th',
 '<table><th',
 '<table><th',
 '<table><th',
 '<table><th',
 '<table><th']

In [388]:
[split.page_content for split in all_splits if split.metadata['page_number'] == 3]

['Employees You are eligible for benefits on the 1st of the month following date of hire (or date of hire if hired on the 1st of the month), if you work at least 20 hours per week. You must enroll within 31 days of your hire date or date of eligibility. If you miss this deadline, you will need to wait until the next Open Enrollment period (which usually takes place in the fall) unless you experience a Qualifying Life Event during the year (as explained to the right).',
 'Dependents You are also allowed to enroll eligible dependents in some of the LP plans. Documentation is required for any eligible dependent covered under LP health plans such as a marriage license or birth certificate. Eligibility depends on what benefits you are enrolling in. Eligible dependents include:',
 '· Your legal spouse;',
 '· Your child under age 26 (regardless of financial dependency, residency with you, marital status, or student status), or if older, your unmarried child who is principally supported by you

In [329]:
[split.page_content for split in all_splits if split.category == "Table"]

AttributeError: 'Document' object has no attribute 'category'

In [372]:
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


In [373]:


retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

rag_chain = (
    {"context": docsearch | format_docs, 
     "question": RunnablePassthrough()
     }
    | prompt
    | gpt_4o
    | StrOutputParser()
)


question = "When am I eligible for benefits?"
rag_chain.invoke(question)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The provided context does not contain specific information about eligibility criteria for benefits. Therefore, I cannot determine when you are eligible for benefits based on the given text.'

In [378]:

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | gpt_4o
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}).assign(answer=rag_chain_from_docs)

question = "When can I enroll in benefits??"
rag_chain_with_source.invoke(question)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'context': [Document(page_content='When Do Benefits End?', metadata={'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'filetype': 'application/pdf', 'page_number': 3}),
  Document(page_content='When Do Benefits End?', metadata={'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'filetype': 'application/pdf', 'page_number': 3}),
  Document(page_content='When Do Benefits End?', metadata={'category': 'Title', 'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 3, 'url': 'some_url'}),
  Document(page_content='When Do Benefits End?', metadata={'category': 'Title', 'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'page_number': 3, 'url': 'some_url'}),
  Document(page_content='When Do Benefits End?', metadata={'filename': '2024-LP-Benefits-Guide---US-Salary-updated-V3.pdf', 'filetype': 'application/pdf', 'page_number':

In [177]:
elements[0].metadata.to_dict().pop('languages', None)

['eng']

In [387]:
[element.metadata['page_number'] for element in elements 
 #if element.metadata.to_dict()['page_number'] == 3
 ]

TypeError: 'ElementMetadata' object is not subscriptable

In [240]:
set(element.category for element in elements 
 #if element.category == "Table"
 )

{'FigureCaption',
 'Image',
 'ListItem',
 'NarrativeText',
 'PageBreak',
 'Table',
 'Title',
 'UncategorizedText'}

In [336]:
table_elements = [element for element in elements if element.category == "Table"]

In [341]:
[element.metadata.text_as_html
    #element.text
 for element in elements if element.category == "Table"][0]

'<table><tr><td>Qualifying Event</td><td>Coverage Change Allowed</td><td>Documentation Required</td></tr><tr><td>Marriage\n @</td><td>Add or remove coverage for employee and/ or dependents, change plans or tiers</td><td>Copy of marriage certificate</td></tr><tr><td>Birth or ) Adoption</td><td>Add coverage for employee and/or new child</td><td>Copy of birth certificate or adoption paperwork</td></tr><tr><td>Loss or Gain of Other + Coverage</td><td>Add or remove coverage for employee and/ or affected dependent</td><td>Documentation providing proof of loss or gain of coverage with effective date</td></tr></table>'

In [342]:
[element.text
 for element in elements if element.category == "Table"][0]

'Qualifying Event Coverage Change Allowed Documentation Required Marriage Birth or Adoption Add or remove coverage for employee and/ or dependents, change plans or tiers Add coverage for employee and/or new child Copy of marriage certificate Copy of birth certificate or adoption paperwork Loss or Gain of Other Coverage Add or remove coverage for employee and/ or affected dependent Documentation providing proof of loss or gain of coverage with effective date'

In [337]:
table_elements[]